In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import numpy as np                   # advanced math library
import keras
import matplotlib.pyplot as plt      # MATLAB like plotting routines

import mlflow
import mlflow.keras

import os
# You can use 'tensorflow', 'torch' or 'jax' as backend. Make sure to set the environment variable before importing.
os.environ["KERAS_BACKEND"] = "tensorflow"
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
plt.imshow(x_train[2], cmap='gray')
y_train[2]
# Again, do some formatting
# we do not flatten each image into a 784-length vector because we want to perform convolutions first

# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

# change integers to 32-bit floating point numbers
x_train = x_train.astype('float32')         
x_test = x_test.astype('float32')

# normalize each value for each pixel for the entire vector for each input
x_train /= 255                              
x_test /= 255

print("Training matrix shape", x_train.shape)
print("Testing matrix shape", x_test.shape)

print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = keras.Sequential(
        [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation="softmax"),
        ]
        )


model.summary()
# Define your hyperparameters
batch_size = 128
epochs = 6

mlflow.set_experiment("MNIST_Classification")

mlflow.keras.autolog()
    
# Train your Keras model

model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"])
    
history = model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_split=0.1,
    )

model.save('mnist_model.keras')
m1 = keras.models.load_model('mnist_model.keras')
sample_index = 11  # Change this index to select a different sample
input_image = np.expand_dims(x_test[sample_index], axis=0)  # Adding a batch dimension

# Perform inference
predictions = m1.predict(input_image)

# Convert the predicted probabilities to class labels
predicted_class = np.argmax(predictions)

# Print the predicted class
print("Predicted class:", predicted_class)
plt.imshow(x_test[11], cmap='gray')
